<h1 align=center><font size = 5>Convolution Variants</h1 >

<h3>Objective for this Notebook</h3>
<h5> 1. 了解不同的卷積變體，在程式如何使用</h5>


# Table of Contents
<li><a href="#ref0">標準卷積（Standard Convolution）</a></li>
<li><a href="#ref1">1x1卷積（1x1 Convolution）</a></li>
<li><a href="#ref2">動態卷積（Dynamic Convolution）</a></li>
<li><a href="#ref3">轉置卷積（Transposed Convolution）</a></li>
<li><a href="#ref4">膨脹卷積（Dilated Convolution）</a></li>
<li><a href="#ref5">空間可分離卷積（Spatially Separable Convolution）</a></li>
<li><a href="#ref6">深度可分離卷積（Depthwise Separable Convolution）</a></li>
<li><a href="#ref7">展平卷積（Flattened Convolution）</a></li>
<li><a href="#ref8">分組卷積（Grouped Convolution）</a></li>
<li><a href="#ref9">混洗分組卷積（Shuffled Grouped Convolution）</a></li>
<hr>

<a id="ref0"></a>
<h1 align=center><font size = 5>標準卷積（Standard Convolution）</h1 >

In [65]:
import torch
import torch.nn as nn

class StandardConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size)

    def forward(self, x):
        return self.conv(x)


In [66]:
input_tensor = torch.randn(1, 3, 224, 224)
std_conv = StandardConv(3, 64, 3)
output_std = std_conv(input_tensor)
print("StandardConv output shape:", output_std.shape)  # 預期輸出: torch.Size([1, 64, 222, 222])

StandardConv output shape: torch.Size([1, 64, 222, 222])


<a id="ref1"></a>
<h1 align=center><font size = 5>1x1卷積（1x1 Convolution）</h1 >

In [67]:
class OneByOneConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


In [68]:
one_by_one_conv = OneByOneConv(3, 64)
output_1x1 = one_by_one_conv(input_tensor)
print("OneByOneConv output shape:", output_1x1.shape)  # 預期輸出: torch.Size([1, 64, 224, 224])

OneByOneConv output shape: torch.Size([1, 64, 224, 224])


<a id="ref2"></a>
<h1 align=center><font size = 5>動態卷積（Dynamic Convolution）</h1 >

In [69]:
class DynamicConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels, out_channels * in_channels * kernel_size * kernel_size, kernel_size=1)

    def forward(self, x):
        batch_size, _, h, w = x.size()
        kernel = self.conv(x)
        kernel = kernel.view(batch_size, self.out_channels, self.in_channels * self.kernel_size * self.kernel_size, h, w)
        x_unfolded = torch.nn.functional.unfold(x, self.kernel_size, padding=(self.kernel_size-1)//2)
        x_unfolded = x_unfolded.view(batch_size, self.in_channels * self.kernel_size * self.kernel_size, h * w)
        kernel = kernel.view(batch_size, self.out_channels, self.in_channels * self.kernel_size * self.kernel_size, h * w)
        output = torch.einsum('boip,bip->bop', kernel, x_unfolded)
        output = output.view(batch_size, self.out_channels, h, w)
        return output

In [70]:
dynamic_conv = DynamicConv(3, 64, 3)
output_dynamic = dynamic_conv(input_tensor)
print("DynamicConv output shape:", output_dynamic.shape)  # 預期輸出: torch.Size([1, 64, 224, 224])

DynamicConv output shape: torch.Size([1, 64, 224, 224])


<a id="ref3"></a>
<h1 align=center><font size = 5>轉置卷積（Transposed Convolution）</h1 >

In [71]:
class TransposedConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=2):
        super().__init__()
        self.conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride)

    def forward(self, x):
        return self.conv(x)


In [72]:
transposed_conv = TransposedConv(3, 64, 2, stride=2)
output_transposed = transposed_conv(input_tensor)
print("TransposedConv output shape:", output_transposed.shape)  # 預期輸出: torch.Size([1, 64, 448, 448])

TransposedConv output shape: torch.Size([1, 64, 448, 448])


<a id="ref4"></a>
<h1 align=center><font size = 5>膨脹卷積（Dilated Convolution）</h1 >

In [73]:
class DilatedConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, dilation=dilation)

    def forward(self, x):
        return self.conv(x)


In [74]:
dilated_conv = DilatedConv(3, 64, 3, dilation=2)
output_dilated = dilated_conv(input_tensor)
print("DilatedConv output shape:", output_dilated.shape)  # 預期輸出: torch.Size([1, 64, 220, 220])

DilatedConv output shape: torch.Size([1, 64, 220, 220])


<a id="ref5"></a>
<h1 align=center><font size = 5>空間可分離卷積（Spatially Separable Convolution）</h1 >

In [75]:
class SpatiallySeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_h = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 3), padding=(0, 1))
        self.conv_w = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 1), padding=(1, 0))

    def forward(self, x):
        x_h = self.conv_h(x)
        x_w = self.conv_w(x)
        return x_h + x_w


In [76]:
separable_conv = SpatiallySeparableConv(3, 64)
output_separable = separable_conv(input_tensor)
print("SpatiallySeparableConv output shape:", output_separable.shape)  # 預期輸出: torch.Size([1, 64, 224, 224])

SpatiallySeparableConv output shape: torch.Size([1, 64, 224, 224])


<a id="ref6"></a>
<h1 align=center><font size = 5>深度可分離卷積（Depthwise Separable Convolution）</h1 >

In [77]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


In [78]:
depthwise_conv = DepthwiseSeparableConv(3, 64)
output_depthwise = depthwise_conv(input_tensor)
print("DepthwiseSeparableConv output shape:", output_depthwise.shape)  # 預期輸出: torch.Size([1, 64, 224, 224])

DepthwiseSeparableConv output shape: torch.Size([1, 64, 224, 224])


<a id="ref7"></a>
<h1 align=center><font size = 5>展平卷積（Flattened Convolution）</h1 >

In [79]:
# 修正後的 FlattenedConv 類別
class FlattenedConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=1)  # 修正輸入通道

    def forward(self, x):
        x_flat = x.view(x.size(0), x.size(1), -1)  # 形狀: (batch_size, in_channels, H*W)
        output = self.conv1d(x_flat)
        return output

In [80]:
flattened_conv = FlattenedConv(3, 64, 3)
input_flattened = input_tensor  # 不需手動展平，交給 forward
output_flattened = flattened_conv(input_flattened)
print("FlattenedConv output shape:", output_flattened.shape)  # 預期輸出: torch.Size([1, 64, 224*224])

FlattenedConv output shape: torch.Size([1, 64, 50176])


<a id="ref8"></a>
<h1 align=center><font size = 5>分組卷積（Grouped Convolution）</h1 >

In [81]:
class GroupedConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, groups=groups)

    def forward(self, x):
        return self.conv(x)


In [82]:
grouped_conv = GroupedConv(3, 63, 3, groups=3)
output_grouped = grouped_conv(input_tensor)
print("GroupedConv output shape:", output_grouped.shape)  # 預期輸出: torch.Size([1, 63, 222, 222])

GroupedConv output shape: torch.Size([1, 63, 222, 222])


<a id="ref9"></a>
<h1 align=center><font size = 5>混洗分組卷積（Shuffled Grouped Convolution）</h1 >

In [83]:
class ShuffledGroupedConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, groups):
        super().__init__()
        assert in_channels % groups == 0 and out_channels % groups == 0
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, groups=groups)
        self.groups = groups

    def forward(self, x):
        x = self.conv(x)
        batch_size, channels, height, width = x.size()
        channels_per_group = channels // self.groups
        x = x.view(batch_size, self.groups, channels_per_group, height, width)
        x = torch.transpose(x, 1, 2).contiguous()
        x = x.view(batch_size, channels, height, width)
        return x


In [84]:
shuffled_conv = ShuffledGroupedConv(3, 63, 3, groups=3)
output_shuffled = shuffled_conv(input_tensor)
print("ShuffledGroupedConv output shape:", output_shuffled.shape)  # 預期輸出: torch.Size([1, 63, 222, 222])

ShuffledGroupedConv output shape: torch.Size([1, 63, 222, 222])
